In [2]:
from typing import List, Union, Dict, Set, Tuple
import numpy as np

### Task 1: Regular expressions
In this task you are asked to create regular expressions that meet the specified conditions. Please use the ``re`` package for the following subtasks.

< < The tasks has been removed as they are too simple to shpwcase onto github > >

### Task 2: Finding the most similar word
The goal of this task is, given a corpus, to find the most similar word for a provided word. As an example we will consider the King James Bible that is is included in the ``gutenberg`` corpus and we are looking to find the word that is most similar to ``god``. We consider two words similar if they appear in the same word context.

__a) Cleaning the input__

Write a function that, given a list of tokens, returns a list of tokens that we consider valid for our task. Moreover, all input tokens should be converted to lower case.

An *invalid* token is a token that 
- is a punctuation mark (consider `string.punctuation`).
- is entirely numeric digits (e.g. `"123"`)
- optionally, if `remove_stopwords=True` then stopwords in the English language are also not considered valid tokens (use nltk's stopwords). 

Use the function signature specified in the cell below.

In [11]:
import nltk
from nltk.corpus.reader.util import StreamBackedCorpusView
import string

def get_valid_tokens(tokens: Union[List[str], StreamBackedCorpusView], remove_stopwords: bool=False) -> List[str]:
    """
    :param tokens: list of tokens that should be cleaned
    :param remove_stopwords: bool indicating if stopwords should be removed
                             False by default
    :return: list of valid tokens
    """
    valids=[]
    for token in tokens:
      token=token.lower()
      if (remove_stopwords==True): #check if stopwords should be removed (True)
        if(token not in stop): #check if the token is a stop word
          if(not token.isdigit()): #check if the token is numeric
            if(token not in string.punctuation): #check if the token is a punctuation mark
              valids.append(token)
      else: #stopwords should not be removed so we repeat just the 2 other checks
        if(not token.isdigit()):
            if(token not in string.punctuation):
              valids.append(token)
    return valids


good. rather than nesting if statements could `continue` to skip current token when condition is meat. better readability

def get_valid_tokens(tokens: Union[List[str], StreamBackedCorpusView], remove_stopwords: bool=False) -> List[str]:
    valid = []
    punct = string.punctuation
    stop = stopwords.words('english')
    digit = re.compile(r"\d+")

    for t in tokens:
        if t in punct:
            continue
        if remove_stopwords and t.lower() in stop:
            continue
        if re.fullmatch(digit, t):
            continue
        valid.append(t.lower())
    return valid

__b) Counting the surroundings__

In our simple model of word similarity we consider words as similar if they are being used in the same context (i.e. they have similar words surrounding them). 

Implement a function that, given a list of words, returns the count of all words in a designated word's surrounding. The context size indicates how many words to the left and right we consider, i.e. a context size of 1 indicates that we only consider the words immediately before and after a central word to be in its context. 

Your function should return a dictionary which maps each word $w$ from the input list to a dictionary. The inner dictionary should map each word that appears in the context of the central word $w$ to a number that indicates how often it appears in the context of $w$.

Make sure your implementation has linear complexity in the vocabulary / corpus length. Use the function signature specified in the cell below.

__Hint:__ Instead of the inner dictionary you can alternatively use `Counter` or `defaultdict` which can be found in the Python module `collections`. Moreover, consider the function ``ngrams`` from ``nltk``.

__Example:__ _For the input_
`['hi', 'james', 'how', 'are', 'you', 'hello', 'catherine', 'how', 'are', 'you']` _and_ `context_size=1`
_we wish to obtain:_
```
{'hi': {'james': 1},
 'james': {'hi': 1, 'how': 1},
 'how': {'james': 1, 'are': 2, 'catherine': 1},
 'are': {'how': 2, 'you': 2},
 'you': {'are': 2, 'hello': 1},
 'hello': {'you': 1, 'catherine': 1},
 'catherine': {'hello': 1, 'how': 1}}
```
__Explanation:__ _The word_ `hi` _is only surrounded by_ `james`_. The word_ `james` _is surrounded by_ `hi` _and_ `how` _. The word_ `how` _is surrounded by_ `james`_, by_ `catherine` _and by_ `are` _twice, ..._

_For_ `contextsize=2` _we obtain:_
```
{'hi': {'james': 1, 'how': 1},
'james': {'hi': 1, 'how': 1, 'are': 1},
'how': {'are': 2, 'you': 2, 'hi': 1, 'james': 1, 'hello': 1, 'catherine': 1},
'are': {'how': 2, 'you': 2, 'james': 1, 'hello': 1, 'catherine': 1},
'you': {'how': 2, 'are': 2, 'hello': 1, 'catherine': 1},
'hello': {'are': 1, 'you': 1, 'catherine': 1, 'how': 1},
'catherine': {'you': 1, 'hello': 1, 'how': 1, 'are': 1}}
```

In [12]:
def get_surrounding_counts(tokens: Union[List[str], StreamBackedCorpusView], context_size: int) -> Dict[str, Dict[str, int]]:
    """
    :param tokens: list of tokens
    :param context_size: integer that indicates the number of context words that are considered on both sides of the central word
    :return: dict of dicts that holds the count of context words for each input token
    """
    full = {}
    for i in set(tokens):
        temp={}
#         print(i)
        for k in [j for j,x in enumerate(tokens) if x==i]:
            for count in range(1,context_size+1):
                ind = count + k
                if ind < len(tokens) :    
                    if tokens[ind] in temp:
                        temp[tokens[ind]]+=1
                    else:
                        temp[tokens[ind]]=1
                ind = k - count
                if ind >=0:
                    if tokens[ind] in temp:
                        temp[tokens[ind]]+=1
                    else:
                        temp[tokens[ind]]=1
        full[i]=temp
    return full

seems to give correct result but too inefficient. consider using collections.counter and looping over nltk ngrams

from nltk.util import ngrams
n_grams = ngrams(tokens, 2*context_size+1, pad_left=True, pad_right=True) # padding: count context words at start/end of list

for gram in n_grams:
....


__c) Keeping the top $k$ words in context__

To reduce the size of our result from task __b)__, we will only consider the most frequent context words for each token. Therefore, write a function that keeps only the $k$ most frequent words in the context of a designated word. Ties are resolved in lexicographic order (e.g. _**A**braham_ would be preferred to _**B**ethlehem_). The function should return a dictionary that maps each word in the outer dictionary to a __set__ of their top $k$ most frequent context words.

In [13]:
def to_sets(context_dict: Dict[str, Dict[str, int]], k: int) -> Dict[str, Set[str]]:
    """
    :param context_dict: dict of dicts that holds the count of context words for each word
    :param k: integer that specifies how many context words should be kept
    :return: dict that maps each word to a set of its k most frequent context words
    """
    sorted_ct={}
    for key,v in context_dict.items():
        sorted_ct[key] = set(dict(sorted(v.items(), key=lambda item: (-item[1],item[0]))[:int(k)]).keys())
    return sorted_ct

__d) Finding the most similar words__

Given a dictionary as obtained in task __c)__ and a word $w$, we want to find the words that have the highest similarity to $w$ in terms of their context. We operationalize context similarity with the Jaccard index (https://en.wikipedia.org/wiki/Jaccard_index).
The Jaccard index of two sets $A$ and $B$ (in our case the sets of context words) is defined as:

$J(A, B) = \frac{|A \cap B|}{|A \cup B|}$

Write a function that returns the words that have the highest similarity to an input word $w$ (ignoring the input word itself). Since several words may have the same Jaccard similarity to the input word, your function should return the set of words that are most similar to $w$ and the respective value of the Jaccard similarity. Use the function signature specified in the cell below.

In [14]:
def find_most_similar_words(input_word: str, contexts: Dict[str, Set[str]]) -> tuple[Set[str], float]:
    """
    :param input_word: string that represents the word we are interested in
    :param contexts: dict that maps each word to a set of its most frequent context words
    :returns:
        - set of the most similar words to the input word
        - float that indicates the highest Jaccard similarity to the input word
    """
    input_set = contexts.pop(input_word, None)
    dis_set={}
    if input_set:
        input_list = list(input_set)
        for word,word_set in contexts.items():
            word_list= list(word_set)
            union = list(set(word_list) | set(input_list))
            intersection = list(set(word_list) & set(input_list))
            distance = len(intersection)/len(union)
            if distance in dis_set:
                dis_set[distance].add(word)
            else:
                dis_set[distance]={word}
        sorted_set = dict(sorted(dis_set.items(), key=lambda item: -item[0]))
        jac_dis = list(sorted_set.keys())[0]
        return (sorted_set[jac_dis],jac_dis)
    else:
        print("Word not present")
        return (0,0)
    return

__e) Bringing it all together__

Finally, we want to apply our functions to the King James Bible (`'bible-kjv.txt'`) that is part of the ``gutenberg`` corpus. We intend to find the word(s) that is (are) most similar to ``god``. Follow the steps below:

i) Obtain a list of all tokens from the King James Bible and store it in a variable ``tokens``.  

ii) Clean the list of tokens with your function from __a)__ to get the list of valid tokens (without removing stopwords) and store it in a variable ``valid_tokens``.  

iii) Apply your function from __b)__ to count the context words for all valid tokens with a ``context_size`` of 2 and store the result in a variable ``context_counts``.  

iv) Using your function from __c)__, keep only the 20 most frequent words in a valid tokens context and store the result in a variable ``context_words``.  

v) Finally, find the most similar words to the word _god_ with your function from __d)__ and store the set of most similar words in the variable ``set_god`` and the highest Jaccard similarity in the variable ``sim_god``.  

In [15]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/parzival/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [16]:
words = nltk.corpus.gutenberg.words('bible-kjv.txt')
print(len(words))
valid_tokens = get_valid_tokens(words,False)
print(len(valid_tokens))

1010654
792050


In [ ]:
#the following cell takes about 5 mins to run

context_counts = get_surrounding_counts(valid_tokens,2)
# print(context_counts)
context_words = to_sets(context_counts,20)
# print(context_words)
set_god,sim_god = find_most_similar_words("god",context_words)
print(set_god,sim_god)